# 2020

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from xml.dom import minidom
import pandas as pd
import numpy as np

In [3]:
jsonObj = pd.read_json(path_or_buf='/content/drive/MyDrive/Touche/touche-task2-passages-version-002.jsonl', lines=True)


In [4]:
jsonObj["old_id"]=jsonObj["id"].str.split("___",expand=True)[0]

In [ ]:
jsonObj.head()

In [6]:
documents_2020=jsonObj.groupby(by="old_id")["contents"].apply(" ".join).reset_index()
documents_2020.head()

,old_id,contents
0,clueweb12-0000tw-00-14115,Do Asian-Americans Face Bias in Admissions at ...
1,clueweb12-0000tw-00-15738,Big Data Analytics a Key Enabler for Social CR...
2,clueweb12-0000tw-00-17905,Best Bets: Fern brings magic to Milford - Delm...
3,clueweb12-0000tw-01-02230,Python Data Analysis Library — pandas: Python ...
4,clueweb12-0000tw-01-15084,MILLARWORLD WEEK: Mark Millar’s Master Plan | ...


In [7]:
rel_2020 = pd.read_csv('/content/drive/MyDrive/Touche/touche2020-task2-relevance-withbaseline.qrels',
                        sep="\s+", 
                        names=["qid", "no", "doc", "rel"])
rel_2020 = rel_2020.astype(str)
rel_2020.head()

,qid,no,doc,rel
0,1,0,clueweb12-0001wb-05-12311,0
1,1,0,clueweb12-1811wb-62-08424,1
2,1,0,clueweb12-1811wb-62-08423,1
3,1,0,clueweb12-1217wb-47-14048,0
4,1,0,clueweb12-1811wb-62-08425,1


In [8]:
def parse_xml(path):
  answer_list = []
  xmldoc = minidom.parse(path)
  itemlist = xmldoc.getElementsByTagName('topics')
  topic_list = itemlist[0].getElementsByTagName('topic')
  for topic in topic_list:
    tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
    answer_list.append(tuple_for_add)
  parsed=pd.DataFrame(answer_list, columns=["Number","Title"])
  return parsed

In [9]:
topics_2020=parse_xml("/content/drive/MyDrive/Touche/topics-task-2-2020.xml")
topics_2020.head()

,Number,Title
0,1,\nWhat is the difference between sex and love?\n
1,2,"\nWhich is better, a laptop or a desktop?\n"
2,3,"\nWhich is better, Canon or Nikon?\n"
3,4,\nWhat are the best dish detergents?\n
4,5,\nWhat are the best cities to live in?\n


In [ ]:
topics_2020.dtypes
rel_2020.dtypes

In [10]:
percentage=[]
not_common_docs=[]
for qid in topics_2020["Number"]:
  relevant=rel_2020.loc[rel_2020["qid"]==qid]["doc"]
  intersection=set(relevant) & set(documents_2020["old_id"])
  not_common=list(set(relevant) -set(documents_2020["old_id"]))
  percent=len(intersection)/len(relevant)
  not_common_docs.append(not_common)
  percentage.append({"title_id":qid,"Percentage":percent,"Not_common":not_common})

In [11]:
df=pd.DataFrame(percentage)
df.head()

,title_id,Percentage,Not_common
0,1,0.735294,"[clueweb12-1811wb-62-08423, clueweb12-0200wb-7..."
1,2,0.909091,"[clueweb12-1201tw-18-01435, clueweb12-0109wb-2..."
2,3,0.968750,[clueweb12-0500wb-68-25390]
3,4,0.972222,[clueweb12-1700tw-38-15104]
4,5,0.972222,[clueweb12-1514wb-77-21531]


In [12]:
not_common_docs=list(np.concatenate(not_common_docs).flat)

In [13]:
new_rel_2020=rel_2020[~rel_2020["doc"].isin(not_common_docs)]

In [ ]:
new_rel_2020.to_csv("/content/drive/MyDrive/Touche/new_rel_2020.csv")

In [ ]:
documents_2020.to_csv("/content/drive/MyDrive/Touche/docs_2020.csv")

In [14]:
percentage=[]
not_common_docs=[]
for qid in topics_2020["Number"]:
  relevant=new_rel_2020.loc[new_rel_2020["qid"]==qid]["doc"]
  intersection=set(relevant) & set(documents_2020["old_id"])
  not_common=list(set(relevant) -set(documents_2020["old_id"]))
  percent=len(intersection)/len(relevant)
  not_common_docs.append(not_common)
  percentage.append({"title_id":qid,"Percentage":percent,"Not_common":not_common})

In [ ]:
percentage